In [1]:
import pandas as pd
from pathlib import Path
import altair as alt

# Frame = all languages in GB
# k = range from 5 to 500, steps of 5
# distances = all language in GB
df = pd.read_csv("experiments/experiment-gb-500.csv")

plot = (
    alt.Chart(df)
    .mark_line(color="blue")
    .encode(
        x=alt.X(
            "k",
            title="Sample size (k)",
        ),
        y=alt.Y(
            "entropy",
            scale=alt.Scale(domain=[1.0, 1.14]),
            title="Entropy",
        ),
        color="method",
    )
    .configure_axis(labelFontSize=12, labelFont="serif", titleFontSize=16, titleFont="serif")
)
plot

alt.Chart(...)

In [2]:
plot2 = (
    alt.Chart(df[df['k'] <= 100])
    .mark_line(color="blue")
    .encode(
        x=alt.X(
            "k",
            title="Sample size (k)",
        ),
        y=alt.Y(
            "entropy",
            scale=alt.Scale(domain=[1.0, 1.14]),
            title="Entropy",
        ),
        color="method",
    )
    .configure_axis(labelFontSize=12, labelFont="serif", titleFontSize=16, titleFont="serif")
)
plot2

alt.Chart(...)

In [10]:
plot.save("plots/entropy-by-k-plot.pdf")
plot2.save("plots/entropy-by-k-peak-plot.pdf")